In [1]:
import pandas as pd
import numpy as np
#BasisSets/XXX.file
import csv
import re
import os
import glob
import sys
import matplotlib.pyplot as plt
from operator import itemgetter
from datetime import datetime
import seaborn as sns
from mpl_toolkits import mplot3d
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import itertools
from itertools import islice
pd.set_option('display.max_rows', 15)
pd.set_option('display.max_columns', 10)

# Uncontracted pcJ-4 Benchmarking Download

In [271]:
JJBaseValues = pd.DataFrame(columns=['Molecule','basis'])
Data_List = []
MainFolder = 'BaseValuesNoSolvation'
Molecule_List = ['F2','HF','CF4','F3C2H3','F3CNH2','F3C3H','F3C2OF','F3CH','F3CSH','F3CCl','F3C2HCH2','F3C3H5','F3C2H3O','F3COH','F3C2NH4']
for Molecule in Molecule_List: #I could try concat in the Molecule loop? 
    Naming_List = ['Molecule', 'basis']
    for basis in ['pcj-4','pcJ-41','pcj-2','pc-2']:
        count = 0 
        N=0
        numbers = re.compile(r"[+-]?\d+(?:\.\d+)?")
        NumberOfCouplings = [] #The Fucks the point of having this number if you use it for nothing?
        NucleusF = []

        with open(f'./Ramps/{MainFolder}/{basis}/{Molecule}/{basis}_{Molecule}.out') as f:
            counter=0
            IMADUMBIDEA =[]
            ShouldZero = [Molecule, basis]
            for line in f:

                if ' NUCLEUS A = F    ' in line:
                    NucleusF.append(list(map(float, numbers.findall(line))))
                if ' NUCLEUS A = H    ' in line:
                    NucleusF.append(list(map(float, numbers.findall(line))))
                if 'Total spin-spin coupling tensor  (Hz):' in line:
                    F_Total = []
                    for i in range(0,len(NucleusF)):

                        Sub_F = []
                        for j in range(0,len(NucleusF[0])):
                            if Molecule == 'HF':
                                if NucleusF[i][j] == 0:
                                    Sub_F.append(str(int(NucleusF[i][j]))+'F')
                                    F_Total.append(Sub_F)
                                if NucleusF[i][j] == 1:
                                    Sub_F.append(str(int(NucleusF[i][j]))+'H')
                                    F_Total.append(Sub_F)
                            if Molecule == 'F3C2OF':
                                Sub_F.append(str(int(NucleusF[i][j]))+'F')
                                F_Total.append(Sub_F)
                            if Molecule == 'CF4':
                                Sub_F.append(str(int(NucleusF[i][j]))+'F')
                                F_Total.append(Sub_F)

                            if Molecule not in ['CF4','HF','F3C2OF']:
                                if NucleusF[i][j] <= 3:
                                    Sub_F.append(str(int(NucleusF[i][j]))+'F')
                                    F_Total.append(Sub_F)
                                else:
                                    Sub_F.append(str(int(NucleusF[i][j]))+'H')
                                    F_Total.append(Sub_F)

                    Correct_Nucleus = []
                    for i in range(len(F_Total)):
                        #print(F_Total)
                        try:
                            Correct_Nucleus.append(F_Total[2*i])
                        except:
                            continue
                if counter > 0:
                    counter -= 1
                    IMADUMBIDEA.append(line)
                    try:
                        result = re.search('iso=(.*)\n', line,flags=0)
                        #print(result.group(1))
                        ShouldZero.append(float(result.group(1)))
                        #print(ShouldZero)
                    except:
                        continue
                if 'Total spin-spin coupling tensor  (Hz):' in line:  
                    counter = 13
                    #print(line)
        IamAVeryDumbIdea = ['Molecule','basis']
        for i in range(0,len(Correct_Nucleus)):
            IamAVeryDumbIdea.append(f'{Correct_Nucleus[i][0]}:{Correct_Nucleus[i][1]} ssDSO')
            IamAVeryDumbIdea.append(f'{Correct_Nucleus[i][0]}:{Correct_Nucleus[i][1]} ssPSO')
            IamAVeryDumbIdea.append(f'{Correct_Nucleus[i][0]}:{Correct_Nucleus[i][1]} ssFC')
            IamAVeryDumbIdea.append(f'{Correct_Nucleus[i][0]}:{Correct_Nucleus[i][1]} ssSD')
            IamAVeryDumbIdea.append(f'{Correct_Nucleus[i][0]}:{Correct_Nucleus[i][1]} ssSD/FC')
            IamAVeryDumbIdea.append(f'{Correct_Nucleus[i][0]}:{Correct_Nucleus[i][1]}')

#                 Intermediate_Frame[f'{Nucleus[i][0]}:{Nucleus[i][1]}'] = [Coupling_Saved[i]]
#                 Intermediate_Frame[f'{Nucleus[i][0]}:{Nucleus[i][1]} ssDSO'] = [ssDSO_Saved[i]]
#                 Intermediate_Frame[f'{Nucleus[i][0]}:{Nucleus[i][1]} ssPSO'] = [ssPSO_Saved[i]]
#                 Intermediate_Frame[f'{Nucleus[i][0]}:{Nucleus[i][1]} ssFC'] = [ssFC_Saved[i]]
#                 Intermediate_Frame[f'{Nucleus[i][0]}:{Nucleus[i][1]} ssSD'] = [ssSD_Saved[i]]
#                 Intermediate_Frame[f'{Nucleus[i][0]}:{Nucleus[i][1]} ssSD/FC'] = [ssSDFC_Saved[i]]             
        #print(FirstRamp,len(ShouldZero)) 
        #print(IamAVeryDumbIdea)
        Intermediate_Frame = pd.DataFrame([ShouldZero],columns=IamAVeryDumbIdea)            
        #print(FirstRamp, Correct_Nucleus, ShouldZero)            

        JJBaseValues = pd.concat([JJBaseValues, Intermediate_Frame], axis=0,ignore_index=True)#print(IamAVeryDumbIdea)

JJBaseValuesFinal = JJBaseValues
JJBaseValuesFinal

,Molecule,basis,0F:1F ssDSO,0F:1F ssPSO,0F:1F ssFC,...,9H:10H ssPSO,9H:10H ssFC,9H:10H ssSD,9H:10H ssSD/FC,9H:10H
0,F2,pcj-4,2.174,8440.231,-342.254,...,NaN,NaN,NaN,NaN,NaN
1,F2,pcJ-41,2.173,8439.708,-376.055,...,NaN,NaN,NaN,NaN,NaN
2,F2,pcj-2,-0.007,5416.253,-25.769,...,NaN,NaN,NaN,NaN,NaN
3,F2,pc-2,-0.170,-134.301,3.332,...,NaN,NaN,NaN,NaN,NaN
4,HF,pcj-4,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
55,F3COH,pc-2,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN
56,F3C2NH4,pcj-4,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN
57,F3C2NH4,pcJ-41,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN
58,F3C2NH4,pcj-2,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN


In [272]:
JJBaseValuesFinal.to_csv('./ResultsDataFrames/pcJ4UncontractedComponentsALLBASIS.csv', na_rep='NaN', index=False)

# Single Ramp (pcJ-4) Download

In [274]:
JJF3C2H3 = pd.DataFrame(columns=['Molecule','basis','First Ramp'])
Data_List = []
MainFolder = 'SingleRamps4to11'
Molecule_List = ['F2','HF','CF4','F3C2H3','F3CNH2','F3C3H','F3C2OF','F3CH','F3CSH','F3CCl','F3C2HCH2','F3C3H5','F3C2H3O','F3COH','F3C2NH4']
for Molecule in Molecule_List: #I could try concat in the Molecule loop? 
    Naming_List = ['Molecule', 'basis', 'First Ramp']
    for basis in ['pcj-4','pcj-2','pc-2']:
        count = 0 
        N=0
        for FirstRamp in [5,6,7,8,9,10,11]:
            numbers = re.compile(r"[+-]?\d+(?:\.\d+)?")
            NumberOfCouplings = [] #The Fucks the point of having this number if you use it for nothing?
            NucleusF = []
            
            with open(f'./Ramps/{MainFolder}/{basis}/{FirstRamp}/{Molecule}/{basis}_{FirstRamp}_{Molecule}.out') as f:
                counter=0
                IMADUMBIDEA =[]
                ShouldZero = [Molecule, basis, FirstRamp]
                for line in f:
                    
                    if ' NUCLEUS A = F    ' in line:
                        NucleusF.append(list(map(float, numbers.findall(line))))
                    if ' NUCLEUS A = H    ' in line:
                        NucleusF.append(list(map(float, numbers.findall(line))))
                    if 'Total spin-spin coupling tensor  (Hz):' in line:
                        F_Total = []
                        for i in range(0,len(NucleusF)):

                            Sub_F = []
                            for j in range(0,len(NucleusF[0])):
                                if Molecule == 'HF':
                                    if NucleusF[i][j] == 0:
                                        Sub_F.append(str(int(NucleusF[i][j]))+'F')
                                        F_Total.append(Sub_F)
                                    if NucleusF[i][j] == 1:
                                        Sub_F.append(str(int(NucleusF[i][j]))+'H')
                                        F_Total.append(Sub_F)
                                if Molecule == 'F3C2OF':
                                    Sub_F.append(str(int(NucleusF[i][j]))+'F')
                                    F_Total.append(Sub_F)
                                if Molecule == 'CF4':
                                    Sub_F.append(str(int(NucleusF[i][j]))+'F')
                                    F_Total.append(Sub_F)

                                if Molecule not in ['CF4','HF','F3C2OF']:
                                    if NucleusF[i][j] <= 3:
                                        Sub_F.append(str(int(NucleusF[i][j]))+'F')
                                        F_Total.append(Sub_F)
                                    else:
                                        Sub_F.append(str(int(NucleusF[i][j]))+'H')
                                        F_Total.append(Sub_F)
                                    
                        Correct_Nucleus = []
                        for i in range(len(F_Total)):
                            #print(F_Total)
                            try:
                                Correct_Nucleus.append(F_Total[2*i])
                            except:
                                continue
                    if counter > 0:
                        counter -= 1
                        IMADUMBIDEA.append(line)
                        try:
                            result = re.search('iso=(.*)\n', line,flags=0)
                            #print(result.group(1))
                            ShouldZero.append(float(result.group(1)))
                            #print(ShouldZero)
                        except:
                            continue
                    if 'Total spin-spin coupling tensor  (Hz):' in line:  
                        counter = 13
                        #print(line)
            IamAVeryDumbIdea = ['Molecule','basis','First Ramp']
            for i in range(0,len(Correct_Nucleus)):
                IamAVeryDumbIdea.append(f'{Correct_Nucleus[i][0]}:{Correct_Nucleus[i][1]} ssDSO')
                IamAVeryDumbIdea.append(f'{Correct_Nucleus[i][0]}:{Correct_Nucleus[i][1]} ssPSO')
                IamAVeryDumbIdea.append(f'{Correct_Nucleus[i][0]}:{Correct_Nucleus[i][1]} ssFC')
                IamAVeryDumbIdea.append(f'{Correct_Nucleus[i][0]}:{Correct_Nucleus[i][1]} ssSD')
                IamAVeryDumbIdea.append(f'{Correct_Nucleus[i][0]}:{Correct_Nucleus[i][1]} ssSD/FC')
                IamAVeryDumbIdea.append(f'{Correct_Nucleus[i][0]}:{Correct_Nucleus[i][1]}')

            Intermediate_Frame = pd.DataFrame([ShouldZero],columns=IamAVeryDumbIdea)            
            #print(FirstRamp, Correct_Nucleus, ShouldZero)            

            JJF3C2H3 = pd.concat([JJF3C2H3, Intermediate_Frame], axis=0,ignore_index=True)#print(IamAVeryDumbIdea)

JJBaseValuesF3C2H3 = JJF3C2H3
JJBaseValuesF3C2H3

,Molecule,basis,First Ramp,0F:1F ssDSO,0F:1F ssPSO,...,9H:10H ssPSO,9H:10H ssFC,9H:10H ssSD,9H:10H ssSD/FC,9H:10H
0,F2,pcj-4,5,2.133,8373.630,...,NaN,NaN,NaN,NaN,NaN
1,F2,pcj-4,6,2.132,8383.880,...,NaN,NaN,NaN,NaN,NaN
2,F2,pcj-4,7,2.144,8387.210,...,NaN,NaN,NaN,NaN,NaN
3,F2,pcj-4,8,2.159,8397.862,...,NaN,NaN,NaN,NaN,NaN
4,F2,pcj-4,9,2.171,8420.227,...,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
310,F3C2NH4,pc-2,7,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN
311,F3C2NH4,pc-2,8,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN
312,F3C2NH4,pc-2,9,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN
313,F3C2NH4,pc-2,10,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN


In [268]:
JJBaseValuesF3C2H3.to_csv('./ResultsDataFrames/SingleRampResultsComponentsALLBASIS.csv', na_rep='NaN', index=False)

In [277]:
JJF3C2H3 = pd.DataFrame(columns=['Molecule','basis','First Ramp', 'Second Ramp'])
Data_List = []
MainFolder = 'DoubleRamps4to11'
Molecule_List = ['F2','HF','CF4','F3C2H3','F3CNH2','F3C3H','F3C2OF','F3CH','F3CSH','F3CCl','F3C2HCH2','F3C3H5','F3C2H3O','F3COH','F3C2NH4']
for Molecule in Molecule_List: #I could try concat in the Molecule loop? 
    Naming_List = ['Molecule', 'basis', 'First Ramp', 'Second Ramp']
    for basis in ['pcj-4','pcj-2','pc-2']:
        count = 0 
        N=0
        euy1 = []
        for FirstRamp in [5,6,7,8,9,10,11]:
            
            for SecondRamp in [5,6,7,8,9,10,11]:
                                
                euy1.append(f'({FirstRamp},{SecondRamp})')

                if FirstRamp == SecondRamp:
                    continue

                if f'({SecondRamp},{FirstRamp})' in euy1:
                    continue
                numbers = re.compile(r"[+-]?\d+(?:\.\d+)?")
                NumberOfCouplings = [] #The Fucks the point of having this number if you use it for nothing?
                NucleusF = []

                with open(f'./Ramps/{MainFolder}/{basis}/{FirstRamp}/{SecondRamp}/{Molecule}/{basis}_{FirstRamp}_{SecondRamp}_{Molecule}.out') as f:
                    counter=0
                    IMADUMBIDEA =[]
                    ShouldZero = [Molecule, basis, FirstRamp, SecondRamp]
                    for line in f:

                        if ' NUCLEUS A = F    ' in line:
                            NucleusF.append(list(map(float, numbers.findall(line))))
                        if ' NUCLEUS A = H    ' in line:
                            NucleusF.append(list(map(float, numbers.findall(line))))
                        if 'Total spin-spin coupling tensor  (Hz):' in line:
                            F_Total = []
                            for i in range(0,len(NucleusF)):

                                Sub_F = []
                                for j in range(0,len(NucleusF[0])):
                                    if Molecule == 'HF':
                                        if NucleusF[i][j] == 0:
                                            Sub_F.append(str(int(NucleusF[i][j]))+'F')
                                            F_Total.append(Sub_F)
                                        if NucleusF[i][j] == 1:
                                            Sub_F.append(str(int(NucleusF[i][j]))+'H')
                                            F_Total.append(Sub_F)
                                    if Molecule == 'F3C2OF':
                                        Sub_F.append(str(int(NucleusF[i][j]))+'F')
                                        F_Total.append(Sub_F)
                                    if Molecule == 'CF4':
                                        Sub_F.append(str(int(NucleusF[i][j]))+'F')
                                        F_Total.append(Sub_F)

                                    if Molecule not in ['CF4','HF','F3C2OF']:
                                        if NucleusF[i][j] <= 3:
                                            Sub_F.append(str(int(NucleusF[i][j]))+'F')
                                            F_Total.append(Sub_F)
                                        else:
                                            Sub_F.append(str(int(NucleusF[i][j]))+'H')
                                            F_Total.append(Sub_F)

                            Correct_Nucleus = []
                            for i in range(len(F_Total)):
                                #print(F_Total)
                                try:
                                    Correct_Nucleus.append(F_Total[2*i])
                                except:
                                    continue
                        if counter > 0:
                            counter -= 1
                            IMADUMBIDEA.append(line)
                            try:
                                result = re.search('iso=(.*)\n', line,flags=0)
                                #print(result.group(1))
                                ShouldZero.append(float(result.group(1)))
                                #print(ShouldZero)
                            except:
                                continue
                        if 'Total spin-spin coupling tensor  (Hz):' in line:  
                            counter = 13
                            #print(line)
                IamAVeryDumbIdea = ['Molecule','basis','First Ramp','Second Ramp']
                for i in range(0,len(Correct_Nucleus)):
                    IamAVeryDumbIdea.append(f'{Correct_Nucleus[i][0]}:{Correct_Nucleus[i][1]} ssDSO')
                    IamAVeryDumbIdea.append(f'{Correct_Nucleus[i][0]}:{Correct_Nucleus[i][1]} ssPSO')
                    IamAVeryDumbIdea.append(f'{Correct_Nucleus[i][0]}:{Correct_Nucleus[i][1]} ssFC')
                    IamAVeryDumbIdea.append(f'{Correct_Nucleus[i][0]}:{Correct_Nucleus[i][1]} ssSD')
                    IamAVeryDumbIdea.append(f'{Correct_Nucleus[i][0]}:{Correct_Nucleus[i][1]} ssSD/FC')
                    IamAVeryDumbIdea.append(f'{Correct_Nucleus[i][0]}:{Correct_Nucleus[i][1]}')

                Intermediate_Frame = pd.DataFrame([ShouldZero],columns=IamAVeryDumbIdea)            
                #print(FirstRamp, Correct_Nucleus, ShouldZero)            

                JJF3C2H3 = pd.concat([JJF3C2H3, Intermediate_Frame], axis=0,ignore_index=True)#print(IamAVeryDumbIdea)

JJBaseValuesF3C2H3 = JJF3C2H3
JJBaseValuesF3C2H3

,Molecule,basis,First Ramp,Second Ramp,0F:1F ssDSO,...,9H:10H ssPSO,9H:10H ssFC,9H:10H ssSD,9H:10H ssSD/FC,9H:10H
0,F2,pcj-4,5,6,2.124,...,NaN,NaN,NaN,NaN,NaN
1,F2,pcj-4,5,7,2.166,...,NaN,NaN,NaN,NaN,NaN
2,F2,pcj-4,5,8,2.178,...,NaN,NaN,NaN,NaN,NaN
3,F2,pcj-4,5,9,2.179,...,NaN,NaN,NaN,NaN,NaN
4,F2,pcj-4,5,10,2.177,...,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
940,F3C2NH4,pc-2,8,10,NaN,...,NaN,NaN,NaN,NaN,NaN
941,F3C2NH4,pc-2,8,11,NaN,...,NaN,NaN,NaN,NaN,NaN
942,F3C2NH4,pc-2,9,10,NaN,...,NaN,NaN,NaN,NaN,NaN
943,F3C2NH4,pc-2,9,11,NaN,...,NaN,NaN,NaN,NaN,NaN


In [278]:
JJBaseValuesF3C2H3.to_csv('./ResultsDataFrames/DoubleRampResultsComponentsALLBASIS.csv', na_rep='NaN', index=False)

In [286]:
DestroyedBasisSets = pd.DataFrame(columns=['Molecule','basis'])
Data_List = []
MainFolder = 'MessypcJ4'
Molecule_List = ['F2','HF','CF4','F3C2H3','F3CNH2','F3C3H','F3C2OF','F3CH','F3CSH','F3CCl','F3C2HCH2','F3C3H5','F3C2H3O','F3COH','F3C2NH4']
for Molecule in Molecule_List: #I could try concat in the Molecule loop? 
    Naming_List = ['Molecule', 'basis', 'First Ramp', 'Second Ramp']
    for basis in ['pcJ-4-1','pcJ-4-2','pcJ-4-3','pcJ-4-4','pcJ-4-5','pcJ-4-6','pcJ-4-7','pcJ-4-8','pcJ-4-1p','pcJ-4-2p','pcJ-4-d','pcJ-4-f','pcJ-4.fake']:
        count = 0 
        N=0

        numbers = re.compile(r"[+-]?\d+(?:\.\d+)?")
        NumberOfCouplings = [] #The Fucks the point of having this number if you use it for nothing?
        NucleusF = []

        with open(f'./Ramps/{MainFolder}/{basis}/{Molecule}/{basis}_{Molecule}.out') as f:
            counter=0
            IMADUMBIDEA =[]
            ShouldZero = [Molecule, basis]
            for line in f:

                if ' NUCLEUS A = F    ' in line:
                    NucleusF.append(list(map(float, numbers.findall(line))))
                if ' NUCLEUS A = H    ' in line:
                    NucleusF.append(list(map(float, numbers.findall(line))))
                if 'Total spin-spin coupling tensor  (Hz):' in line:
                    F_Total = []
                    for i in range(0,len(NucleusF)):

                        Sub_F = []
                        for j in range(0,len(NucleusF[0])):
                            if Molecule == 'HF':
                                if NucleusF[i][j] == 0:
                                    Sub_F.append(str(int(NucleusF[i][j]))+'F')
                                    F_Total.append(Sub_F)
                                if NucleusF[i][j] == 1:
                                    Sub_F.append(str(int(NucleusF[i][j]))+'H')
                                    F_Total.append(Sub_F)
                            if Molecule == 'F3C2OF':
                                Sub_F.append(str(int(NucleusF[i][j]))+'F')
                                F_Total.append(Sub_F)
                            if Molecule == 'CF4':
                                Sub_F.append(str(int(NucleusF[i][j]))+'F')
                                F_Total.append(Sub_F)

                            if Molecule not in ['CF4','HF','F3C2OF']:
                                if NucleusF[i][j] <= 3:
                                    Sub_F.append(str(int(NucleusF[i][j]))+'F')
                                    F_Total.append(Sub_F)
                                else:
                                    Sub_F.append(str(int(NucleusF[i][j]))+'H')
                                    F_Total.append(Sub_F)

                    Correct_Nucleus = []
                    for i in range(len(F_Total)):
                        #print(F_Total)
                        try:
                            Correct_Nucleus.append(F_Total[2*i])
                        except:
                            continue
                if counter > 0:
                    counter -= 1
                    IMADUMBIDEA.append(line)
                    try:
                        result = re.search('iso=(.*)\n', line,flags=0)
                        #print(result.group(1))
                        ShouldZero.append(float(result.group(1)))
                        #print(ShouldZero)
                    except:
                        continue
                if 'Total spin-spin coupling tensor  (Hz):' in line:  
                    counter = 13
                    #print(line)
        IamAVeryDumbIdea = ['Molecule','basis']
        for i in range(0,len(Correct_Nucleus)):
            IamAVeryDumbIdea.append(f'{Correct_Nucleus[i][0]}:{Correct_Nucleus[i][1]} ssDSO')
            IamAVeryDumbIdea.append(f'{Correct_Nucleus[i][0]}:{Correct_Nucleus[i][1]} ssPSO')
            IamAVeryDumbIdea.append(f'{Correct_Nucleus[i][0]}:{Correct_Nucleus[i][1]} ssFC')
            IamAVeryDumbIdea.append(f'{Correct_Nucleus[i][0]}:{Correct_Nucleus[i][1]} ssSD')
            IamAVeryDumbIdea.append(f'{Correct_Nucleus[i][0]}:{Correct_Nucleus[i][1]} ssSD/FC')
            IamAVeryDumbIdea.append(f'{Correct_Nucleus[i][0]}:{Correct_Nucleus[i][1]}')

        Intermediate_Frame = pd.DataFrame([ShouldZero],columns=IamAVeryDumbIdea)            
        #print(FirstRamp, Correct_Nucleus, ShouldZero)            

        DestroyedBasisSets = pd.concat([DestroyedBasisSets, Intermediate_Frame], axis=0,ignore_index=True)#print(IamAVeryDumbIdea)

DestroyedBasisSetsFinal = DestroyedBasisSets
DestroyedBasisSetsFinal

,Molecule,basis,0F:1F ssDSO,0F:1F ssPSO,0F:1F ssFC,...,9H:10H ssPSO,9H:10H ssFC,9H:10H ssSD,9H:10H ssSD/FC,9H:10H
0,F2,pcJ-4-1,2.173,8437.931,-279.423,...,NaN,NaN,NaN,NaN,NaN
1,F2,pcJ-4-2,2.092,8471.144,-205.330,...,NaN,NaN,NaN,NaN,NaN
2,F2,pcJ-4-3,2.048,8285.818,-212.215,...,NaN,NaN,NaN,NaN,NaN
3,F2,pcJ-4-4,1.593,8044.043,-168.408,...,NaN,NaN,NaN,NaN,NaN
4,F2,pcJ-4-5,0.372,6170.362,-79.730,...,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
190,F3C2NH4,pcJ-4-1p,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN
191,F3C2NH4,pcJ-4-2p,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN
192,F3C2NH4,pcJ-4-d,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN
193,F3C2NH4,pcJ-4-f,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN


In [287]:
DestroyedBasisSetsFinal.to_csv('./ResultsDataFrames/DestroyedpcJ4All2.csv', na_rep='NaN', index=False)